
# 线性回归的简洁实现

随着深度学习框架的发展，开发深度学习应用变得越来越便利。实践中，我们通常可以用比上一节更简洁的代码来实现同样的模型。在本节中，我们将介绍如何使用tensorflow2.0推荐的keras接口更方便地实现线性回归的训练。

## 生成数据集

我们生成与上一节中相同的数据集。其中`features`是训练数据特征，`labels`是标签。

In [2]:
import tensorflow as tf
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = tf.random.normal([num_examples, num_inputs],mean=0,stddev=1)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += tf.random.normal(labels.shape,0,1)
print(features[0], labels[0])

tf.Tensor([-0.36099175  0.8862902 ], shape=(2,), dtype=float32) tf.Tensor(1.5807515, shape=(), dtype=float32)


虽然tensorflow2.0对于线性回归可以直接拟合，不用再划分数据集，但我们仍学习一下读取数据的方法

In [3]:
dataset=tf.data.Dataset.from_tensor_slices((features,labels))
train_db = tf.data.Dataset.from_tensor_slices((features, labels)).batch(10)
for(x,y) in train_db:
    print(x,y)

tf.Tensor(
[[-0.36099175  0.8862902 ]
 [-0.1839203   0.69210005]
 [ 0.9376471  -1.8326032 ]
 [-0.8767882   0.06162997]
 [ 2.0317926   1.1839459 ]
 [ 0.14049357 -0.06233747]
 [ 0.2712547  -1.0112672 ]
 [-0.9887581   1.0038339 ]
 [-1.5709177   0.6849357 ]
 [ 1.4573355   0.14634162]], shape=(10, 2), dtype=float32) tf.Tensor(
[ 1.5807515   2.0043135  11.38282     1.9067247   3.3788707   4.174027
  9.1142845  -0.3445133   0.72452116  6.381167  ], shape=(10,), dtype=float32)
tf.Tensor(
[[ 0.67836106  0.21218857]
 [ 0.7805339   0.1751083 ]
 [-0.40067694  0.01032858]
 [-0.5874898  -0.23388049]
 [ 1.7244262   0.12885976]
 [-0.36420974  0.8306901 ]
 [-0.67727995 -0.42827734]
 [ 0.32643402 -0.31216484]
 [-1.6838305   0.02156664]
 [ 0.9449985   0.92415   ]], shape=(10, 2), dtype=float32) tf.Tensor(
[5.158974  5.8439374 2.4597278 4.726387  7.7726755 1.1911871 6.674436
 5.978803  2.1197748 4.261552 ], shape=(10,), dtype=float32)
tf.Tensor(
[[ 0.45266736  0.5006    ]
 [-1.8639725   1.0549592 ]
 [-0.5

 14.098777    0.1638473   0.06691381  7.3574424 ], shape=(10,), dtype=float32)
tf.Tensor(
[[ 0.81126904 -1.1031327 ]
 [ 0.5773093   0.52569664]
 [-2.5689397  -0.28584072]
 [-0.29423556  1.5898315 ]
 [ 1.1268957   0.1845486 ]
 [ 1.4647071  -0.39989224]
 [ 0.9653343  -0.6262987 ]
 [ 0.9070358  -0.7697269 ]
 [ 1.5318714   0.27064866]
 [ 0.9391578  -1.9667487 ]], shape=(10, 2), dtype=float32) tf.Tensor(
[10.896067    2.937134    1.5807672  -0.21874225  5.876279    8.186303
  8.1127615   8.3542185   6.0143247  12.638846  ], shape=(10,), dtype=float32)
tf.Tensor(
[[ 1.3963196  -1.8400693 ]
 [-0.10845938 -0.20753965]
 [-1.4031105  -0.05314368]
 [ 0.5519065   0.26290342]
 [-0.801027   -0.67163175]
 [ 0.25558716 -0.5031781 ]
 [ 0.35773808 -0.45481104]
 [-0.7326419  -0.77669924]
 [ 0.6297      1.558593  ]
 [ 1.9685175  -0.99267465]], shape=(10, 2), dtype=float32) tf.Tensor(
[12.289181   4.5058174  2.6665316  5.46426    4.1114993  4.9977365
  7.4312325  5.9680605  1.509115   9.973228 ], shape=(10

定义模型,tensorflow 2.0推荐使用keras定义网络，故使用keras定义网络
我们先定义一个模型变量`model`，它是一个`Sequential`实例。
在keras中，`Sequential`实例可以看作是一个串联各个层的容器。
在构造模型时，我们在该容器中依次添加层。
当给定输入数据时，容器中的每一层将依次计算并将输出作为下一层的输入。
重要的一点是，在keras中我们无须指定每一层输入的形状。
因为为线性回归，输入层与输出层全连接，故定义一层

In [4]:
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.Dense(1))

定义损失函数和优化器：损失函数为mse，优化器选择sgd随机梯度下降
在keras中，定义完模型后，调用`compile()`方法可以配置模型的损失函数和优化方法。定义损失函数只需传入`loss`的参数，keras定义了各种损失函数，并直接使用它提供的平方损失`mse`作为模型的损失函数。同样，我们也无须实现小批量随机梯度下降，只需传入`optimizer`的参数，keras定义了各种优化算法，我们这里直接指定学习率为0.01的小批量随机梯度下降`tf.keras.optimizers.SGD(0.03)`为优化算法

In [5]:
model.compile(optimizer=tf.keras.optimizers.SGD(0.01),
              loss='mse')

在使用keras训练模型时，我们通过调用`model`实例的`fit`函数来迭代模型。`fit`函数只需传入你的输入x和输出y，还有epoch遍历数据的次数，每次更新梯度的大小batch_size, 这里定义epoch=3，batch_size=10。
使用keras甚至完全不需要去划分数据集

In [6]:
model.fit(features,labels,epochs=3, batch_size=10)

Train on 1000 samples
Epoch 1/3
1000/1000 [==============================] - 1s 1ms/sample - loss: 9.5898
Epoch 2/3
1000/1000 [==============================] - 0s 224us/sample - loss: 1.1697
Epoch 3/3
1000/1000 [==============================] - 0s 296us/sample - loss: 0.9868


下面我们分别比较学到的模型参数和真实的模型参数。我们可以通过model的`get_weights()`来获得其权重（`weight`）和偏差（`bias`）。学到的参数和真实的参数很接近。

In [12]:
true_w, model.get_weights()[0]

([2, -3.4], array([[ 1.9412669],
        [-3.3460715]], dtype=float32))

In [13]:
true_b, model.get_weights()[1]

(4.2, array([4.2120686], dtype=float32))